In [25]:
lines = readlines("input.txt") .|> l -> split(l, "") .|> x -> parse(Int, x)

100-element Vector{Vector{Int64}}:
 [4, 5, 6, 7, 8, 9, 4, 3, 0, 1  …  7, 8, 9, 5, 9, 8, 6, 5, 5, 5]
 [3, 4, 5, 8, 9, 6, 3, 2, 1, 2  …  8, 9, 3, 4, 8, 9, 5, 4, 3, 4]
 [2, 3, 4, 8, 9, 5, 4, 3, 2, 9  …  5, 6, 9, 5, 6, 7, 9, 4, 2, 3]
 [1, 2, 3, 7, 8, 9, 5, 4, 9, 8  …  6, 7, 8, 9, 7, 9, 8, 9, 1, 2]
 [3, 4, 5, 6, 7, 9, 7, 9, 8, 7  …  7, 8, 9, 9, 9, 8, 7, 8, 9, 3]
 [4, 5, 6, 7, 9, 8, 9, 8, 9, 6  …  8, 9, 9, 9, 8, 7, 6, 7, 8, 9]
 [5, 6, 7, 9, 8, 7, 8, 7, 8, 9  …  9, 9, 9, 8, 8, 6, 5, 6, 7, 8]
 [8, 7, 9, 8, 7, 6, 5, 6, 7, 8  …  8, 9, 8, 7, 6, 5, 4, 5, 6, 8]
 [9, 9, 8, 7, 6, 5, 4, 6, 8, 8  …  7, 8, 9, 6, 5, 4, 3, 4, 5, 6]
 [2, 3, 9, 8, 5, 4, 3, 5, 6, 7  …  6, 9, 8, 7, 4, 3, 2, 3, 4, 5]
 [1, 9, 8, 7, 6, 5, 2, 3, 5, 6  …  9, 8, 9, 8, 5, 4, 1, 2, 3, 4]
 [0, 1, 9, 8, 6, 2, 1, 2, 3, 7  …  8, 7, 8, 9, 2, 1, 0, 1, 2, 3]
 [9, 2, 3, 9, 4, 3, 2, 3, 4, 8  …  9, 6, 5, 4, 3, 2, 1, 2, 3, 4]
 ⋮
 [6, 7, 9, 8, 7, 6, 5, 4, 5, 6  …  8, 9, 6, 5, 4, 4, 5, 7, 9, 9]
 [7, 9, 9, 9, 8, 7, 4, 3, 4, 3  …  9, 8, 7, 8, 5, 5,

In [26]:
function is_in_bounds(row, col, grid)
    return (1 ≤ row ≤ length(grid)) && (1 ≤ col ≤ length(grid[1]))
end

function get_neighbours(row, col, grid)
    neighbours = Int[]
    for deltas in [(-1, 0), (1, 0), (0, -1), (0, 1)]
        neigh_row = row + deltas[1]
        neigh_col = col + deltas[2]
        if is_in_bounds(neigh_row, neigh_col, grid)
            push!(neighbours, grid[neigh_row][neigh_col])
        end
    end
    return neighbours
end

get_neighbours (generic function with 1 method)

In [27]:
function get_risk_levels(grid)
    risk_levels = Int[]
    for row in 1:length(grid)
        for col in 1:length(grid[1])
            elem = grid[row][col]
            if all(elem .< get_neighbours(row, col, grid))
                push!(risk_levels, elem + 1)
            end
        end
    end
    return risk_levels
end

get_risk_levels (generic function with 1 method)

In [28]:
part1 = sum(get_risk_levels(lines))

425

In [29]:
function basin_new_neighbours!(row, col, grid, visited)
    neighbours = Tuple{Int, Int}[]
    for deltas in [(-1, 0), (1, 0), (0, -1), (0, 1)]
        neigh_row = row + deltas[1]
        neigh_col = col + deltas[2]
        neigh = (neigh_row, neigh_col)
        if is_in_bounds(neigh_row, neigh_col, grid) && neigh ∉ visited
            push!(visited, neigh)
            if grid[neigh_row][neigh_col] < 9
                push!(neighbours, neigh)
            end
        end
    end
    return neighbours
end

function get_new_neighbours!(points, grid, visited)
    neighbours = Tuple{Int, Int}[]
    for point in points
        append!(neighbours, basin_new_neighbours!(point[1], point[2], grid, visited))
    end
    return neighbours
end

function get_basin!(row, col, grid, visited)
    push!(visited, (row, col))
    if grid[row][col] == 9
       return [] 
    end

    basin = [(row, col)]
    neighbours = basin_new_neighbours!(row, col, grid, visited)
    append!(basin, neighbours)
    while length(neighbours) != 0
        neighbours = get_new_neighbours!(neighbours, grid, visited)
        append!(basin, neighbours)
    end
    return basin
end

function get_basins(grid)
    basins = []
    remaining = Set([(row, col) for row in 1:length(grid) for col in 1:length(grid[1])])
    visited = Set{Tuple{Int, Int}}()
    while length(remaining) > 0
        cell = first(remaining)
        push!(basins, get_basin!(cell[1], cell[2], grid, visited))
        remaining = setdiff(remaining, visited)
    end
    return basins
end

function part2(grid)
    basins_length = get_basins(grid) .|> length
    sorted_length = sort(basins_length, rev=true)
    return sorted_length[1] * sorted_length[2] * sorted_length[3]
end

part2 (generic function with 1 method)

In [30]:
part2(lines)

1135260